In [42]:
import s3fs
import sys
sys.path.append('../util')
import util_data_access as da
from tqdm.auto import tqdm
import numpy as np
import os, glob
import pandas as pd 

# Download Files

In [3]:
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': 'http://s3.dev.obdc.bcs.bloomberg.com'})

In [5]:
!mkdir nyt_output

In [6]:
for f in tqdm(fs.ls('aspangher/edit-pathways/spark_processing_scripts-output/nyt/')):
    f = '/'.join(f.split('/')[1:])
    fname = f.split('/')[-1]
    
    da.download_file('nyt_output/' + fname, f)

  0%|          | 0/785 [00:00<?, ?it/s]

In [7]:
da.download_file('nyt.db', 'edit-pathways/dbs/newssniffer-nytimes.db')

True

In [8]:
fs.ls('aspangher/edit-pathways/spark_processing_scripts-output_sentences/nyt')

['aspangher/edit-pathways/spark_processing_scripts-output_sentences/nyt/df_nyt__start_0__end_20000__num_1.pkl']

# Read Files

In [9]:
dfs = []
for f in tqdm(glob.glob('nyt_output/*')):
    df = pd.read_csv(f, compression=None, index_col=0)
    dfs.append(df)

In [10]:
full_diffs_df = pd.concat(dfs)

In [11]:
full_diffs_df[['entry_id', 'version_x', 'version_y']].drop_duplicates().shape

(53033, 3)

In [8]:
full_diffs_df[['entry_id', 'version_x', 'version_y']].drop_duplicates().shape

(49904, 3)

In [12]:
full_diffs_df.head()

,entry_id,version_x,version_y,sent_idx_x,sent_idx_y,avg_sentence_distance_x,avg_sentence_distance_y
0,566535,2,3,NaN,50.0,NaN,NaN
1,565936,1,2,27.0,27.0,0.0,0.0
2,566126,0,1,210.0,210.0,0.0,0.0
3,566639,0,1,50.0,50.0,0.0,0.0
4,566157,3,4,NaN,34.0,NaN,NaN


In [13]:
(full_diffs_df
 .assign(c=1)
 .groupby(['entry_id', 'version_x', 'version_y', 'sent_idx_x', 'sent_idx_y'])['c']
 .sum()
 .value_counts()
)

1    2021505
Name: c, dtype: int64

In [14]:
(full_diffs_df
 .fillna('nan')
 .drop_duplicates(['entry_id', 'version_x', 'version_y', 'sent_idx_x', 'sent_idx_y'])
 .assign(c=1)
 .groupby(['entry_id', 'version_x', 'version_y', 'sent_idx_x', 'sent_idx_y'])['c']
 .sum()
 .loc[lambda s: s>1]
#  .head()
)

Series([], Name: c, dtype: int64)

In [30]:
(full_diffs_df
 .fillna('nan')
 .drop_duplicates(['entry_id', 'version_x', 'version_y', 'sent_idx_x', 'sent_idx_y'])
 .replace(to_replace='nan', value=np.nan)
 .to_pickle('partial-nyt-output.pkl', compression='gzip')
)

In [31]:
! mv partial-nyt-output.pkl 2021-05-19__partial-nyt-output.pkl

In [34]:
da.upload_file('2021-05-19__partial-nyt-output.pkl', 'edit-pathways/output_for_sheena/2021-05-19__partial-nyt-matched-output.pkl')

True

In [36]:
da.upload_file('nyt_sent_output/df_nyt__start_0__end_20000__num_1.pkl', 'edit-pathways/output_for_sheena/df_nyt__start_0__end_20000__num_1.pkl')

True

In [35]:
ls nyt_sent_output

df_nyt__start_0__end_20000__num_1.pkl


In [33]:
fs.ls('aspangher/edit-pathways')

['aspangher/edit-pathways/csvs',
 'aspangher/edit-pathways/db-dumps',
 'aspangher/edit-pathways/dbs',
 'aspangher/edit-pathways/pkls',
 'aspangher/edit-pathways/pqs',
 'aspangher/edit-pathways/sample_files_for_sheena',
 'aspangher/edit-pathways/spacy',
 'aspangher/edit-pathways/spark_processing_scripts-output',
 'aspangher/edit-pathways/spark_processing_scripts-output_sentences',
 'aspangher/edit-pathways/temp',
 'aspangher/edit-pathways/tmp']

In [22]:
## sentences

In [23]:
mkdir nyt_sent_output

In [24]:
for f in fs.ls('aspangher/edit-pathways/spark_processing_scripts-output_sentences/nyt'):
    f = '/'.join(f.split('/')[1:])
    fname = f.split('/')[-1]
    da.download_file('nyt_sent_output/' + fname , f)

In [25]:
import pandas as pd 
import pickle

# with open('nyt_sent_output/df_nyt__start_0__end_20000__num_1.pkl', 'rb') as f:
#     sent_df = pickle.load(f)
sent_df = pd.read_pickle('nyt_sent_output/df_nyt__start_0__end_20000__num_1.pkl', compression='gzip')

In [41]:
full_diffs_df[['entry_id', 'version_x', 'version_y']].drop_duplicates().shape
full_diffs_df['entry_id'].drop_duplicates().shape

(17999,)

In [26]:
calc_entry_versions = (full_diffs_df
 .set_index('entry_id')[['version_x', 'version_y']]
 .unstack()
 .to_frame('version')
 .reset_index()
 .drop('level_0', axis=1)
 .drop_duplicates()
)

In [29]:
sent_df[['entry_id', 'version']].drop_duplicates().shape

(79292, 2)

In [27]:
(sent_df[['entry_id','version']]
 .drop_duplicates()
 .merge(
     calc_entry_versions, 
     right_on=['entry_id', 'version'], 
     left_on=['entry_id', 'version'], 
     how='inner'
 ).shape
)

(71073, 2)

# Calculate number of expected

In [55]:
import sqlite3

In [56]:
con = sqlite3.connect('nyt.db')

In [99]:
full_df = pd.read_sql('select * from entryversion where num_versions > 1 and num_versions < 40 ', con=con)

In [98]:
eligible_ids = pd.read_sql('select entry_id, version from entryversion where num_versions > 1 and num_versions < 40 ', con=con)

In [64]:
expected = []
for entry_id, versions in eligible_ids.groupby('entry_id').aggregate(list)['version'].iteritems():
    for version_pair in zip(versions[:-1], versions[1:]):
        expected.append({'entry_id': entry_id, 'version_pair': version_pair})

In [66]:
expected_df = pd.DataFrame(expected)

In [68]:
expected_df.shape

(278944, 2)

# Look at fetching operation

In [91]:
sys.path.append('edit-project/spark_processing_scripts')

In [92]:
import util_general as ug

In [93]:
t = ug._download_prefetched_data_csv('nyt', False, True)

INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Resetting dropped connection: s3.dev.obdc.bcs.bloomberg.com


  0%|          | 0/784 [00:00<?, ?it/s]

INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Resetting dropped connection: s3.dev.obdc.bcs.bloomberg.com


In [95]:
t.shape

(2147675, 7)

In [97]:
full_df.shape

(2238973, 7)

In [101]:
df = ug.get_rows_to_process_df(
    500, 0, t, full_df
)

In [3]:
import sys
sys.path.append('../')
import spark_processing_scripts.util_spark as sus
import spark_processing_scripts.util_general as sug

In [4]:
full_db = sug.download_pq_to_df('nyt')

FileNotFoundError: aspangher/edit-pathways/pqs/newssniffer-nytimes.pq

In [47]:
prefetched_df = sug.download_prefetched_data('nyt', split_sentences=False)

In [74]:
t = (prefetched_df
 .set_index('entry_id')[['version_x', 'version_y']]
 .unstack()
 .to_frame('version')
 .reset_index()
 .drop('level_0', axis=1)
 .drop_duplicates()
)

In [88]:
t2 = full_db[['entry_id', 'version']].drop_duplicates()

In [96]:
prefetched_df['entry_id'].shape

(2560285,)

In [100]:
'entry_id' in prefetched_df

True

In [59]:
full_db['entry_id'].drop_duplicates().shape

(20000,)

In [ ]:
df = sug.get_rows_to_process_df(
    500, 0, prefetched_df, full_db
)

In [1]:
import sqlite3
import pandas as pd 
with sqlite3.connect('nyt.db') as con:
    full_nyt_df = pd.read_sql('''
        SELECT * FROM entryversion
        WHERE num_versions > 1
        AND num_versions < 40
    ''', con=con)

In [2]:
full_nyt_df.loc[lambda df: ~df['entry_id'].isin(full_db['entry_id'])].shape

NameError: name 'full_db' is not defined

In [107]:
full_nyt_df.loc[lambda df: df['entry_id'].isin(full_db['entry_id'])].shape

(79294, 12)